In [154]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [228]:
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

# Scratch notebook for handling long stop to stop segments

* goal is to provide additional map detail for rural/express routes

In [3]:
## KART speedmap as displayed on site
itp_id = 148
analysis_date = dt.date(2022, 6, 1)

In [4]:
pbar = tqdm()

0it [00:00, ?it/s]

In [5]:
kart_analysis = rt.OperatorDayAnalysis(itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found parquet
less than 1km of data
vehicle positions gdf must not be empty
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 60 seconds of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 60 seconds of data
less than 60 seconds of data
less than 1km of data
less than 1km of data
less than 1km of data
less t

In [9]:
rt.OperatorDayAnalysis._generate_stop_delay_view(kart_analysis)

In [10]:
kart_analysis.rt_trips >> filter(_.route_short_name=='12')


,calitp_itp_id,calitp_url_number,service_date,trip_key,trip_id,route_id,direction_id,shape_id,calitp_extracted_at,calitp_deleted_at,route_short_name,route_long_name,route_desc,route_type,median_time,direction,mean_speed_mph,calitp_agency_name
266,148,0,2022-06-01,2644978890892601525,949,12,0,11,2021-07-02,2099-01-01,12,Avenal,None,3,12:27:19,Westbound,31.666138,Kings Area Rural Transit
267,148,0,2022-06-01,3494167986599370378,947,12,0,3,2021-07-02,2099-01-01,12,Avenal,None,3,07:26:32,Southbound,38.489930,Kings Area Rural Transit
268,148,0,2022-06-01,-4140891564404050759,950,12,0,27,2021-07-02,2099-01-01,12,Avenal,None,3,17:44:36.500000,Westbound,40.296843,Kings Area Rural Transit


In [11]:
# kart_analysis.debug_dict['950_stopsegs'] >> arrange(_.stop_sequence)

In [12]:
# gdf = kart_analysis.routelines >> filter(_.shape_id == '3')
# shared_utils.rt_utils.map_line(gdf)

In [13]:
#kart_analysis.rt_trips >> filter(_.route_short_name == '2')

In [14]:
# kart_analysis.stop_delay_view = kart_analysis.stop_delay_view >> mutate(last_loc = _.shape_meters.shift(1))

In [15]:
# long_trip = kart_analysis.stop_delay_view >> filter(_.trip_id == '947') >> arrange(_.stop_sequence)

In [16]:
# test_center = (long_trip >> arrange(_.shape_meters)).iloc[10:49, :]

In [17]:
really_fast_bus = kart_analysis.position_interpolators['104']['rt']

In [18]:
# dir(really_fast_bus)

# Mapping a shape

In [60]:
df = (
    tbl.views.gtfs_schedule_dim_shapes()
    >> filter(
        _.calitp_extracted_at <= analysis_date,
        _.calitp_deleted_at > analysis_date,
    )
    >> filter(_.calitp_itp_id == 148, _.shape_id == '49')
    # >> select(_.calitp_itp_id, _.calitp_url_number, _.shape_id)
    >> collect()
)

In [63]:
df = df >> arrange(_.shape_pt_sequence)

In [157]:
import geopandas as gpd
import folium

In [65]:
df >> head(3)

,calitp_itp_id,calitp_url_number,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,calitp_extracted_at,calitp_hash,shape_key,calitp_deleted_at
17,148,0,49,36.325647,-119.652688,1,None,2021-04-15,ru0ixCJbt8tkUUqzC4Aqwg==,-4907651418736819562,2099-01-01
52,148,0,49,36.326609,-119.652883,2,None,2021-04-15,/P1ceA4gkYRAiN3Bz/Cosg==,-8708430766074839456,2099-01-01
42,148,0,49,36.326440,-119.653930,3,None,2021-04-15,GHPi+x3rcs/0x3WY7Cv8Pg==,-6331366513554695594,2099-01-01


In [66]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.shape_pt_lon, df.shape_pt_lat), crs='EPSG:4326')

In [86]:
gdf = gdf >> select(-_.calitp_extracted_at, -_.calitp_deleted_at)

In [72]:
folium.Map?

Init signature:
folium.Map(
    location=None,
    width='100%',
    height='100%',
    left='0%',
    top='0%',
    position='relative',
    tiles='OpenStreetMap',
    attr=None,
    min_zoom=0,
    max_zoom=18,
    zoom_start=10,
    min_lat=-90,
    max_lat=90,
    min_lon=-180,
    max_lon=180,
    max_bounds=False,
    crs='EPSG3857',
    control_scale=False,
    prefer_canvas=False,
    no_touch=False,
    disable_3d=False,
    png_enabled=False,
    zoom_control=True,
    **kwargs,
)
Docstring:     
Create a Map with Folium and Leaflet.js

Generate a base map of given width and height with either default
tilesets or a custom tileset URL. The following tilesets are built-in
to Folium. Pass any of the following to the "tiles" keyword:

    - "OpenStreetMap"
    - "Mapbox Bright" (Limited levels of zoom for free tiles)
    - "Mapbox Control Room" (Limited levels of zoom for free tiles)
    - "Stamen" (Terrain, Toner, and Watercolor)
    - "Cloudmade" (Must pass API key)
    - "Mapb

In [158]:
m = folium.Map(location=[36.32, -119.65], tiles='cartodbpositron')

In [91]:
import branca

In [95]:
colormap = branca.colormap.LinearColormap(['#ffffed', '#202a44'], vmin=gdf.shape_pt_sequence.min(),
                                        vmax=gdf.shape_pt_sequence.max())

In [97]:
colormap(14)

'#e9eaddff'

In [98]:
# branca.colormap.LinearColormap?

In [101]:
folium.GeoJson(gdf >> select(_.shape_pt_sequence,_.geometry),
               style_function=lambda x: {
            "fillColor": colormap(x["properties"]['shape_pt_sequence'])
            # if x["properties"][plot_col] is not None
            # else "gray",
            # "color": "#FFFFFF",
            # "fillOpacity": 0.8,
            # "weight": 0.2,
            },
              ).add_to(m)

In [19]:
# m

In [20]:
len(really_fast_bus.position_gdf) - len(really_fast_bus.cleaned_positions)

45

# Mapping Short Segments

In [45]:
kart_analysis.export_views_gcs()

/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a production setting unless you are able to rewrite your Parquet/Feather files.

To further ignore this warning, you can do: 
import warnings; warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
/home/jovyan/data-analyses/_shared_utils/shared_utils/utils.py:39: UserWarning: this is an initial implementation of Parquet/Feather file support and associated metadata.  This is tracking version 0.1.0 of the metadata specification at https://github.com/geopandas/geo-arrow-spec

This metadata specification does not yet make stability promises.  We do not yet recommend using this in a 

In [46]:
import rt_filter_map_plot

In [123]:
importlib.reload(rt_filter_map_plot)

<module 'rt_filter_map_plot' from '/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py'>

In [51]:
analysis_date

datetime.date(2022, 6, 1)

In [53]:
month_day = analysis_date.strftime('%m_%d')

In [57]:
gdf = gpd.read_parquet(f'{shared_utils.rt_utils.GCS_FILE_PATH}stop_delay_views/{itp_id}_{month_day}.parquet')

In [58]:
gdf >> head(3)

,shape_meters,stop_id,stop_name,geometry,shape_id,trip_key,trip_id,stop_sequence,arrival_time,route_id,route_short_name,direction_id,actual_time,delay_seconds
0,878.449814,109,Lacey @ Carl's Jr.,POINT (30719.208 -187595.241),7,-5.590482e+18,601,2.0,2022-06-01T13:48:19,6,6,0,2022-06-01T13:47:30.381586,0.0
1,1430.878220,110,Lacey @ Government Center,POINT (30168.081 -187596.770),7,-5.590482e+18,601,3.0,2022-06-01T13:50:38,6,6,0,2022-06-01T13:47:42.780581,0.0
2,1878.449814,None,None,None,7,NaN,601,3.5,NaT,None,None,None,2022-06-01T13:47:52.826114,NaN


In [61]:
gdf.arrival_time.iloc[2]

'NaT'

In [65]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

found parquet


In [66]:
rt_day.set_filter(start_time='06:00', end_time='08:00', route_names=['12'])

In [67]:
filtered = rt_day._filter(rt_day.stop_delay_view) >> arrange(_.stop_sequence)

In [68]:
m = rt_day.segment_speed_map()

/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encountered in subtract
/opt/conda/lib/python3.10/site-packages/numpy/lib/function_base.py:4009: RuntimeWarning: invalid value encounte

In [97]:
# m

In [70]:
df = filtered >> arrange(_.stop_sequence)

## Another Operator

In [229]:
itp_id = 300

In [269]:
pbar = tqdm()

0it [00:00, ?it/s]

In [271]:
importlib.reload(rt)

<module 'rt_analysis' from '/home/jovyan/data-analyses/rt_delay/rt_analysis.py'>

In [ ]:
rt_analysis = rt.OperatorDayAnalysis(itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found parquet
could not interpolate segments for shape 25929
could not interpolate segments for shape 25950
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data
less than 1km of data


In [ ]:
rt_analysis.export_views_gcs()

In [ ]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, analysis_date)

In [ ]:
rt_day.set_filter(start_time='06:00', end_time='09:00', route_names=['R10'])

In [ ]:
m = rt_day.segment_speed_map(how='low_speeds')

In [ ]:
m